In [7]:
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair

import json

In [8]:
BDG_ROOT_URL = '35.157.161.130'
BDB_URL_PORT = 9984
bdb = BigchainDB('http://{}:{}'.format(BDG_ROOT_URL, BDB_URL_PORT))

## Creacion de Assets

In [9]:
# Alice es un usuario definido por un par de claves
alice = generate_keypair()

# bob tbn
bob = generate_keypair()

In [10]:
alice_secret = generate_keypair()

In [11]:
# Alice's vote
vote = {
    'data': {
        'vote': {
            # lo que hay aqui se puede cambiar
            'identifier': 'question_14102017',
            'manufacturer': True,
        },
    }, 
}

In [12]:
# Creacion del asset

# create asset
prepared_creation_tx = bdb.transactions.prepare(
        operation='CREATE',
        signers=alice.public_key,
        asset=bicycle,
)

fulfilled_creation_tx = bdb.transactions.fulfill(
    prepared_creation_tx, 
    private_keys=alice.private_key, # signed with private key
)

# send asset
sent_creation_tx = bdb.transactions.send(fulfilled_creation_tx)

txid = fulfilled_creation_tx['id'] # id de la transaccion

NameError: name 'bicycle' is not defined

In [13]:
fulfilled_creation_tx

NameError: name 'fulfilled_creation_tx' is not defined

In [14]:
creation_tx = bdb.transactions.retrieve(txid)

NameError: name 'txid' is not defined

In [15]:
alice.public_key

'Ay4ysq4AEcvop8jRqRYAhiiwabutLEuqLb4mAEyXmDq2'

In [16]:
# Alice is the owner
creation_tx['outputs'][0]['public_keys'][0] == alice.public_key

NameError: name 'creation_tx' is not defined

In [17]:
# Buscar bicis por serial number:
assets = bdb.assets.get(search="pamtumaca")
assets

[{'data': {'sida': {'cuanto_sida': ['mu', 'cho'],
    'identifier': 'pamtumaca',
    'manufacturer': 'bkfab'}},
  'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'}]

## Transferencia de Assets

In [18]:
fulfilled_creation_tx['id']

NameError: name 'fulfilled_creation_tx' is not defined

In [19]:
# Alice le quiere enviar la bici con id
bici = assets[0]
bici['id']

'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'

In [20]:
# Notar que la bici es de alice
creation_tx = bdb.transactions.retrieve(bici['id'])
creation_tx['outputs'][0]['public_keys'][0] == alice.public_key

False

In [21]:
transfer_asset = {
    'id': bici['id'],
}

In [22]:
output_index = 0 # selecion de la opciones de output
output = creation_tx['outputs'][output_index]
transfer_input = {
    'fulfillment': output['condition']['details'],
    'fulfills': {
        'output_index': output_index,
        'transaction_id': creation_tx['id'],
    },
    'owners_before': output['public_keys'],
}

prepared_transfer_tx = bdb.transactions.prepare(
    operation='TRANSFER',
    asset=transfer_asset,
    inputs=transfer_input,
    recipients=alice_secret.public_key,
)

fulfilled_transfer_tx = bdb.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=alice.private_key,
)

MissingPrivateKeyError: A private key is missing!

In [23]:
sent_transfer_tx = bdb.transactions.send(fulfilled_transfer_tx)

NameError: name 'fulfilled_transfer_tx' is not defined

In [24]:
sent_transfer_tx

NameError: name 'sent_transfer_tx' is not defined

In [25]:
bdb.transactions.status('5cfba500c357bef3243e694e4b5afa3c104f4402c86b608df4b8cf5e074185a4')

{'status': 'valid'}

In [26]:
bob.public_key

'AE8kLiACUv3FtYo34bnBKkULZY1GyksMrHQDcENjh5JE'

In [27]:
search = bdb.assets.get(search='pamtumaca')

In [28]:
bdb.transactions.get(asset_id="f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586")

[{'asset': {'data': {'sida': {'cuanto_sida': ['mu', 'cho'],
     'identifier': 'pamtumaca',
     'manufacturer': 'bkfab'}}},
  'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586',
  'inputs': [{'fulfillment': 'pGSAIPFKQ_qXpUSijT_z7M46BYUx0t_zUD7pm4j_1lqBMxbcgUAybT0Fx1YVm_GyqxO0WtpXn1oxDX66WX3IQzHHPoDV9suGGaAOoTyHFWj96t7UAIpDswBGsHmvvKoz1D2I1XAO',
    'fulfills': None,
    'owners_before': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
  'metadata': None,
  'operation': 'CREATE',
  'outputs': [{'amount': '1',
    'condition': {'details': {'public_key': 'HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9',
      'type': 'ed25519-sha-256'},
     'uri': 'ni:///sha-256;LikjEvHldymOvitUiFkBNEWwjkc6ygt4OaO2b5tlEtM?fpt=ed25519-sha-256&cost=131072'},
    'public_keys': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
  'version': '1.0'},
 {'asset': {'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
  'id': '5cfba500c357bef3243e694e4b5afa3c104f4402

In [29]:
bdb.transactions.retrieve('5cfba500c357bef3243e694e4b5afa3c104f4402c86b608df4b8cf5e074185a4')

{'asset': {'id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
 'id': '5cfba500c357bef3243e694e4b5afa3c104f4402c86b608df4b8cf5e074185a4',
 'inputs': [{'fulfillment': 'pGSAIPFKQ_qXpUSijT_z7M46BYUx0t_zUD7pm4j_1lqBMxbcgUA1hBW_JNLF8AcsO6O7Yv72ZnocyVIh8nQvwDanWnRKE-rYEvNeF7IOVtmhLIEKovx-pTlOnusatG_-xbE6n1gH',
   'fulfills': {'output_index': 0,
    'transaction_id': 'f4efbda2a593e47bda56f40ed3c6ba671e36c186eaf6f728159178cf0dbe8586'},
   'owners_before': ['HEtwHeTfCdv1jfDNLBESJCAhSFRUumAP25ExbuAtKQm9']}],
 'metadata': None,
 'operation': 'TRANSFER',
 'outputs': [{'amount': '1',
   'condition': {'details': {'public_key': 'DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW',
     'type': 'ed25519-sha-256'},
    'uri': 'ni:///sha-256;pmwlr6fe-TdVjHEDBlEGwa90VLXvx8isHNevqv_Q2cI?fpt=ed25519-sha-256&cost=131072'},
   'public_keys': ['DcpriYX58N1NqyXBkA7WpsV7FKMwqwhvcdmuSGEQxKYW']}],
 'version': '1.0'}

In [30]:
alice.public_key

'Ay4ysq4AEcvop8jRqRYAhiiwabutLEuqLb4mAEyXmDq2'

In [31]:
bob.public_key

'AE8kLiACUv3FtYo34bnBKkULZY1GyksMrHQDcENjh5JE'